In [7]:
#import libraries, dependencies

#TODO: import CSV and create a data frame
#Using a CSV file with all zip codes in Denver from:
#https://www.downloadexcelfiles.com/us_en/download-list-us-zip-codes-colorado-state#gsc.tab=0
#selected only zip codes in Denver City using excel


#TODO: select parameters for request to geoapify, which will find all healthcare facilities in each zip code(Kelsy)

#TODO: Use a for loop to make an api request for each zip code, and place the healthcare facility results into a DF(Kelsy)
#Question 1: are healthcare facilities more dense where the population is dense? In other words: 
#Is there a correlation between pop of zip code and  number of facilities in the zip code?
#TODO: Draw a map which locates each healthcare facility (color by type of facility? OR color by zip code?size by pop?)(Lisa)
#TODO: scatter plot and linear regression of facility count vs population



#TODO: count # facilities with wheelchair access
#TODO: If not very close to 100%, plot types of facility against wheelchair access(bar)
#TODO: map with color of facility dot by wheelchair access

#TODO: get population, median age, per capita income, of each zip code and add to data frame with healthcare facility info (Lisa)
#from https://data.colorado.gov/
#TODO: plot #of healthcare facilities per zip code against population per zip code (double bars with zip as x axis?)
#TODO: plot #of healthcare facilities per zip code against median income
#TODO: scatter plot and lin reg of  zip code against median income   
#TODO: plot #of healthcare facilities per zip code against per cap income
#TODO: scatter plot and lin reg of  zip code against per cap income 

#TODO: written analysis

#TODO: prepare slides for presentation

#TODO: practice presentation(or at least be clear on respective roles)


In [24]:
import pandas as pd
import numpy as np
import requests
import json
from config import api_key

In [28]:
#TODO: import CSV and create a data frame
#Using a CSV file with all zip codes in Denver from:
#https://www.downloadexcelfiles.com/us_en/download-list-us-zip-codes-colorado-state#gsc.tab=0
#selected only zip codes in Denver City using excel

file = 'resources/denver_zipcodes.csv'
denver_df = file_one_df = pd.read_csv(file)
denver_df


,zip,City,State,County Name
0,80101,AGATE,CO,ELBERT
1,81020,AGUILAR,CO,LAS ANIMAS
2,80720,AKRON,CO,WASHINGTON
3,81101,ALAMOSA,CO,ALAMOSA
4,80510,ALLENSPARK,CO,BOULDER
...,...,...,...,...
606,80864,YODER,CO,EL PASO
607,80759,YUMA,CO,YUMA
608,NaN,NaN,NaN,NaN
609,NaN,NaN,NaN,NaN


In [9]:
#example call to geoapify
url = 'https://api.geoapify.com/v2/places'
latitude = -33.8698439
longitude = 151.2082848
radius = 5000
filters = f'circle: {longitude}, {latitude}', {radius}
bias = f"proximity:{longitude},{latitude}"
limit = 5
parameters = {
    'categories': "healthcare",
    "limit": limit,
    "filters":filters,
    "bias": bias,
    "apiKey":api_key}

results = requests.get(url , params=parameters)    
results.json()       


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Chemist Warehouse',
    'country': 'Australia',
    'country_code': 'au',
    'state': 'New South Wales',
    'city': 'Sydney',
    'municipality': 'Council of the City of Sydney',
    'postcode': '2000',
    'suburb': 'Sydney',
    'street': 'Strand Arcade',
    'lon': 151.2080349,
    'lat': -33.8696525,
    'state_code': 'NSW',
    'formatted': 'Chemist Warehouse, Strand Arcade, Sydney NSW 2000, Australia',
    'address_line1': 'Chemist Warehouse',
    'address_line2': 'Strand Arcade, Sydney NSW 2000, Australia',
    'categories': ['healthcare', 'wheelchair', 'wheelchair.yes'],
    'details': ['details', 'details.facilities'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Chemist Warehouse',
      'brand': 'Chemist Warehou

In [10]:
print(results.url)

https://api.geoapify.com/v2/places?categories=healthcare&limit=5&filters=circle%3A+151.2082848%2C+-33.8698439&filters=5000&bias=proximity%3A151.2082848%2C-33.8698439&apiKey=10df7cccac6c43b08ab640bcf5b7e4a9


In [15]:
#request population data for each zip code from data.colorado.gov

url_pop = 'https://data.colorado.gov/resource/ucnv-vw74.json?'
params_pop = {'zip_code': 80014,
              }
response_pop = restaurant = requests.get(url_pop, params=params_pop)
response = response_pop.json()

In [16]:
response

[{'the_geom': {'type': 'MultiPolygon',
   'coordinates': [[[[-104.880372, 39.642605],
      [-104.880313, 39.643949],
      [-104.880313, 39.645889],
      [-104.880313, 39.645902],
      [-104.880239, 39.645993],
      [-104.880239, 39.646068],
      [-104.880242, 39.646273],
      [-104.880246, 39.64653],
      [-104.880246, 39.647751],
      [-104.880228, 39.647994],
      [-104.880086, 39.650773],
      [-104.880003, 39.650771],
      [-104.878196, 39.650753],
      [-104.877466, 39.650746],
      [-104.877429, 39.650746],
      [-104.877305, 39.650758],
      [-104.877186, 39.650784],
      [-104.877072, 39.650821],
      [-104.876945, 39.650883],
      [-104.876845, 39.650948],
      [-104.876625, 39.651117],
      [-104.876464, 39.651263],
      [-104.876352, 39.651377],
      [-104.876281, 39.651459],
      [-104.875954, 39.651942],
      [-104.875942, 39.651966],
      [-104.877, 39.65305],
      [-104.876557, 39.65305],
      [-104.876323, 39.65305],
      [-104.87619, 39.653

In [22]:
#getting just the pop from the result
pop = response[0]['pop']
pop


'39540'